# Prompt Learning with NeMo

In this example, we utilize NeMo's [prompt learning](https://docs.nvidia.com/deeplearning/nemo/user-guide/docs/en/stable/nlp/nemo_megatron/prompt_learning.html)
feature to showcase how to adapt a large language model (LLM) to 
a downstream task, such as financial sentiment predictions. 

The prompt learning technique shown in the example is [p-tuning](https://arxiv.org/abs/2103.10385), which adds a small prompt encoder network to the LLM
to produce virtual token embeddings that guide the model toward the desired output of the downstream task.

For more details on how to change hyperparameters for prompt learning in NeMo, see this [tutorial](https://github.com/NVIDIA/NeMo/blob/main/tutorials/nlp/Multitask_Prompt_and_PTuning.ipynb) which is also the basis for this NVFlare tutorial.

## Dependencies
We assume you followed the instructions [here](../../README.md#requirements) 
to install the NeMo framework and the NeMo-NVFlare package. 

## Download the pre-trained LLM
In this example, we use a `MegatronGPTModel`, a transformer-based language model based on the GPT architecture.

In [1]:
# Check what GPT .nemo models we have available on NGC
from nemo.collections.nlp.models.language_modeling.megatron_gpt_model import MegatronGPTModel
MegatronGPTModel.list_available_models()

[NeMo W 2023-06-01 17:59:34 experimental:27] Module <class 'nemo.collections.nlp.data.language_modeling.megatron.megatron_batch_samplers.MegatronPretrainingRandomBatchSampler'> is experimental, not ready for production and is not fully supported. Use at your own risk.
[NeMo W 2023-06-01 17:59:34 experimental:27] Module <class 'nemo.collections.nlp.models.text_normalization_as_tagging.thutmose_tagger.ThutmoseTaggerModel'> is experimental, not ready for production and is not fully supported. Use at your own risk.
[NeMo W 2023-06-01 17:59:38 experimental:27] Module <class 'nemo.collections.asr.modules.audio_modules.SpectrogramToMultichannelFeatures'> is experimental, not ready for production and is not fully supported. Use at your own risk.


[PretrainedModelInfo(
 	pretrained_model_name=megatron_gpt_345m,
 	description=345M parameter GPT generative Megatron model.,
 	location=https://api.ngc.nvidia.com/v2/models/nvidia/nemo/megatron_gpt_345m/versions/1/files/megatron_gpt_345m.nemo
 )]

In [2]:
# Download the model from NGC
import os
model_file = "megatron_gpt_345m.nemo"
if not os.path.isfile(model_file):
    !wget "https://api.ngc.nvidia.com/v2/models/nvidia/nemo/megatron_gpt_345m/versions/1/files/$model_file"
else:
    print(f"{model_file} already downloaded.")

megatron_gpt_345m.nemo already downloaded.


## Data preprocessing
As our downstream task, we will use the [Financial PhraseBank dataset](https://huggingface.co/datasets/financial_phrasebank) for sentiment analysis.

The Financial PhraseBank dataset contains the sentiments for financial news headlines from a retail investor's perspective. Further details about the dataset can be found in Malo et al.'s ["Good Debt or Bad Debt: Detecting Semantic Orientations in Economic Texts"](https://arxiv.org/abs/1307.5336).


#### 1. Download the preprocessing scripts
We use the preprocessing scripts provided by NeMo which can be downloaded from GitHub.

In [3]:
script_name = "prompt_learning_financial_phrase_bank_preprocessing.py"
if not os.path.isfile(script_name):
    !wget -N "https://raw.githubusercontent.com/NVIDIA/NeMo/main/scripts/dataset_processing/nlp/financial_phrase_bank/$script_name"
else:
    print(f"{script_name} already downloaded.")

prompt_learning_financial_phrase_bank_preprocessing.py already downloaded.


#### 2. Download the Financial PhraseBank Dataset

Download the `FinancialPhraseBank-v1.0.zip` dataset from [here](https://www.researchgate.net/profile/Pekka_Malo/publication/251231364_FinancialPhraseBank-v1.0/data/0c96051eee4fb1d56e000000/FinancialPhraseBank-v1.0.zip).

Then extract it under `./data`.

#### 3. Preprocess the dataset

In [4]:
!python3 prompt_learning_financial_phrase_bank_preprocessing.py

Saving train split to data/FinancialPhraseBank-v1.0/financial_phrase_bank_train.jsonl
100%|███████████████████████████████████| 1811/1811 [00:00<00:00, 115010.74it/s]
Saving val split to data/FinancialPhraseBank-v1.0/financial_phrase_bank_val.jsonl
100%|█████████████████████████████████████| 226/226 [00:00<00:00, 113604.11it/s]
Saving test split to data/FinancialPhraseBank-v1.0/financial_phrase_bank_test.jsonl
100%|█████████████████████████████████████| 227/227 [00:00<00:00, 122567.84it/s]


#### 4. Split the dataset to simulate clients
Next, we use three clients to simulate federated learning for p-tuning with NeMo.

In [5]:
!python3 data/split_financial_phrase_data.py --data_path data/FinancialPhraseBank-v1.0/financial_phrase_bank_train.jsonl --num_clients 3 --out_dir data/FinancialPhraseBank-v1.0_split

Loaded training data with 1811 entries
Save split 1 of 3 with 604 entries to data/FinancialPhraseBank-v1.0_split/site-1.jsonl
Save split 2 of 3 with 604 entries to data/FinancialPhraseBank-v1.0_split/site-2.jsonl
Save split 3 of 3 with 603 entries to data/FinancialPhraseBank-v1.0_split/site-3.jsonl


## Federated learning simulations
Next, we are using NVFlare's [simulator](https://nvflare.readthedocs.io/en/latest/user_guide/fl_simulator.html) to simulate each client training on their own dataset locally and all three clients training together using the [FedAvg](https://arxiv.org/abs/1602.05629) algorithm implemented in NVFlare.

With this setting, we require a GPU with at least 16GB memory to run all clients in parallel on the same GPU. 
If you have multiple GPUs in your system, you can use the `gpu` argument to assign one GPU for each client, e.g., `gpu="0,1"`.

#### 1. Local P-Tuning
First, we create the configuration files and modify them to include the current directory path to access the dataset and pre-trained LLM.
At this point, we also modify the local number of clients, local epochs and FL rounds to simulate local training.

In [6]:
!python3 create_configs.py --job_folder "jobs/gpt_p-tuning_local_345M" --num_clients 3 --aggregation_epochs 50 --num_rounds 1

Created configs for 3 clients and set ROOT_DIR to /workspace/Code/nvflare/nemo_nvflare/integration/nemo/examples/prompt_learning


Next, simulate each client p-tuning on their local dataset using the FL simulator. To do this, we only run 1 round of FL, with each client running 50 p-tuning epochs on their local dataset.

In [7]:
from nvflare import SimulatorRunner    

simulator = SimulatorRunner(
    job_folder="jobs/gpt_p-tuning_local_345M",
    workspace="/tmp/nvflare/nemo/gpt_p-tuning_local_345M",
    n_clients=3,
    threads=3
)
run_status = simulator.run()
print("Simulator finished with run_status", run_status)

2023-06-01 17:59:42,489 - SimulatorRunner - INFO - Create the Simulator Server.
2023-06-01 17:59:42,495 - Cell - INFO - server: creating listener on tcp://0:37707
2023-06-01 17:59:42,497 - Cell - INFO - server: created backbone external listener for tcp://0:37707
2023-06-01 17:59:42,498 - ConnectorManager - INFO - 2372: Try start_listener Listener resources: {'secure': False, 'host': 'localhost'}
2023-06-01 17:59:42,499 - nvflare.fuel.f3.sfm.conn_manager - INFO - Connector [CH00002 PASSIVE tcp://0:5994] is starting
2023-06-01 17:59:43,002 - Cell - INFO - server: created backbone internal listener for tcp://localhost:5994
2023-06-01 17:59:43,004 - nvflare.fuel.f3.sfm.conn_manager - INFO - Connector [CH00001 PASSIVE tcp://0:37707] is starting
2023-06-01 17:59:43,202 - nvflare.fuel.hci.server.hci - INFO - Starting Admin Server localhost on Port 55737
2023-06-01 17:59:43,203 - SimulatorRunner - INFO - Deploy the Apps.
2023-06-01 17:59:43,215 - SimulatorRunner - INFO - Create the simulate c

[NeMo W 2023-06-01 18:00:01 experimental:27] Module <class 'nemo.collections.nlp.data.language_modeling.megatron.megatron_batch_samplers.MegatronPretrainingRandomBatchSampler'> is experimental, not ready for production and is not fully supported. Use at your own risk.
[NeMo W 2023-06-01 18:00:01 experimental:27] Module <class 'nemo.collections.nlp.models.text_normalization_as_tagging.thutmose_tagger.ThutmoseTaggerModel'> is experimental, not ready for production and is not fully supported. Use at your own risk.
[NeMo W 2023-06-01 18:00:02 experimental:27] Module <class 'nemo.collections.nlp.data.language_modeling.megatron.megatron_batch_samplers.MegatronPretrainingRandomBatchSampler'> is experimental, not ready for production and is not fully supported. Use at your own risk.
[NeMo W 2023-06-01 18:00:02 experimental:27] Module <class 'nemo.collections.nlp.models.text_normalization_as_tagging.thutmose_tagger.ThutmoseTaggerModel'> is experimental, not ready for production and is not fully

2023-06-01 18:00:03,082 - ServerRunner - INFO - [identity=simulator_server, run=simulate_job, wf=share_config, peer=site-3, peer_run=simulate_job, task_name=share_config, task_id=c8a5911c-7e5e-441f-b165-a7c96fbeff7a]: assigned task to client site-3: name=share_config, id=c8a5911c-7e5e-441f-b165-a7c96fbeff7a
2023-06-01 18:00:03,084 - ServerRunner - INFO - [identity=simulator_server, run=simulate_job, wf=share_config, peer=site-3, peer_run=simulate_job, task_name=share_config, task_id=c8a5911c-7e5e-441f-b165-a7c96fbeff7a]: sent task assignment to client. client_name:site-3 task_id:c8a5911c-7e5e-441f-b165-a7c96fbeff7a
2023-06-01 18:00:03,086 - GetTaskCommand - INFO - return task to client.  client_name: site-3  task_name: share_config   task_id: c8a5911c-7e5e-441f-b165-a7c96fbeff7a  sharable_header_task_id: c8a5911c-7e5e-441f-b165-a7c96fbeff7a
2023-06-01 18:00:03,121 - ServerRunner - INFO - [identity=simulator_server, run=simulate_job, wf=share_config, peer=site-3, peer_run=simulate_job]:

I0601 18:00:03.077422 140659256891200 fl_component.py:134] [identity=site-3, run=simulate_job]: Initializing the Learner...
I0601 18:00:03.078066 140659256891200 fl_component.py:134] [identity=site-3, run=simulate_job]: Running with distributed environment: LOCAL_RANK: 0, RANK: 0, WORLD_SIZE 1, MASTER_ADDR: localhost, and MASTER_PORT: 36839
I0601 18:00:03.078348 140659256891200 fl_component.py:134] [identity=site-3, run=simulate_job]: client runner started
I0601 18:00:03.078536 140659256891200 simulator_worker.py:85] Initialize ClientRunner for client: site-3
I0601 18:00:03.088636 140658085013248 communicator.py:200] Received from simulator_server server  (3492 Bytes). getTask: share_config time: 0.007776737213134766 seconds
I0601 18:00:03.092673 140659256891200 fed_client.py:91] pull_task completed. Task name:share_config Status:True 
I0601 18:00:03.092940 140659256891200 fl_component.py:134] [identity=site-3, run=simulate_job, peer=simulator_server, peer_run=simulate_job]: got task a

2023-06-01 18:00:03,494 - ServerRunner - INFO - [identity=simulator_server, run=simulate_job, wf=share_config, peer=site-2, peer_run=simulate_job, task_name=share_config, task_id=c1e25224-6319-49f1-b387-e2bb0c8a2a33]: assigned task to client site-2: name=share_config, id=c1e25224-6319-49f1-b387-e2bb0c8a2a33
2023-06-01 18:00:03,496 - ServerRunner - INFO - [identity=simulator_server, run=simulate_job, wf=share_config, peer=site-2, peer_run=simulate_job, task_name=share_config, task_id=c1e25224-6319-49f1-b387-e2bb0c8a2a33]: sent task assignment to client. client_name:site-2 task_id:c1e25224-6319-49f1-b387-e2bb0c8a2a33
2023-06-01 18:00:03,497 - GetTaskCommand - INFO - return task to client.  client_name: site-2  task_name: share_config   task_id: c1e25224-6319-49f1-b387-e2bb0c8a2a33  sharable_header_task_id: c1e25224-6319-49f1-b387-e2bb0c8a2a33
2023-06-01 18:00:03,536 - ServerRunner - INFO - [identity=simulator_server, run=simulate_job, wf=share_config, peer=site-2, peer_run=simulate_job]:

...

2023-06-01 18:17:56,510 - ServerRunner - INFO - [identity=simulator_server, run=simulate_job, wf=scatter_and_gather, peer=site-2, peer_run=simulate_job]: got result from client site-2 for task: name=train, id=e277103e-8b4a-4a10-9202-cf52a937c773
2023-06-01 18:17:56,513 - ServerRunner - INFO - [identity=simulator_server, run=simulate_job, wf=scatter_and_gather, peer=site-2, peer_run=simulate_job, peer_rc=OK, task_name=train, task_id=e277103e-8b4a-4a10-9202-cf52a937c773]: ignored result submission since server runner's status is done
2023-06-01 18:17:56,515 - SubmitUpdateCommand - INFO - submit_update process. client_name:site-2   task_id:e277103e-8b4a-4a10-9202-cf52a937c773


I0601 18:17:56.414454 140713251325760 fl_component.py:134] [identity=site-2, run=simulate_job, peer=simulator_server, peer_run=simulate_job, task_name=train, task_id=e277103e-8b4a-4a10-9202-cf52a937c773]: Computed 7 weight differences for global model of length 7
I0601 18:17:56.414947 140713251325760 fl_component.py:134] [identity=site-2, run=simulate_job, peer=simulator_server, peer_run=simulate_job, task_name=train, task_id=e277103e-8b4a-4a10-9202-cf52a937c773]: Local steps per epoch: 9
I0601 18:17:56.415169 140713251325760 fl_component.py:134] [identity=site-2, run=simulate_job, peer=simulator_server, peer_run=simulate_job, task_name=train, task_id=e277103e-8b4a-4a10-9202-cf52a937c773]: Local epochs finished. Returning shareable
I0601 18:17:56.416029 140713251325760 fl_component.py:134] [identity=site-2, run=simulate_job, peer=simulator_server, peer_run=simulate_job, task_name=train, task_id=e277103e-8b4a-4a10-9202-cf52a937c773]: finished processing task
I0601 18:17:56.419102 140709


2023-06-01 18:17:56,414 - PromptLearner - INFO - [identity=site-2, run=simulate_job, peer=simulator_server, peer_run=simulate_job, task_name=train, task_id=e277103e-8b4a-4a10-9202-cf52a937c773]: Computed 7 weight differences for global model of length 7
2023-06-01 18:17:56,414 - PromptLearner - INFO - [identity=site-2, run=simulate_job, peer=simulator_server, peer_run=simulate_job, task_name=train, task_id=e277103e-8b4a-4a10-9202-cf52a937c773]: Local steps per epoch: 9
2023-06-01 18:17:56,415 - PromptLearner - INFO - [identity=site-2, run=simulate_job, peer=simulator_server, peer_run=simulate_job, task_name=train, task_id=e277103e-8b4a-4a10-9202-cf52a937c773]: Local epochs finished. Returning shareable
2023-06-01 18:17:56,416 - ClientRunner - INFO - [identity=site-2, run=simulate_job, peer=simulator_server, peer_run=simulate_job, task_name=train, task_id=e277103e-8b4a-4a10-9202-cf52a937c773]: finished processing task
2023-06-01 18:17:56,419 - FederatedClient - INFO - Starting to push 

I0601 18:17:58.548664 140713251325760 fed_client.py:91] pull_task completed. Task name:__end_run__ Status:True 
I0601 18:17:58.549079 140713251325760 fl_component.py:134] [identity=site-2, run=simulate_job, peer=simulator_server, peer_run=simulate_job]: server asked to end the run
I0601 18:17:58.549272 140713251325760 simulator_worker.py:102] End the Simulator run.
I0601 18:17:58.549880 140713251325760 simulator_worker.py:125] Clean up ClientRunner for : site-2 


2023-06-01 18:18:02,196 - MPM - INFO - MPM: Good Bye!
Simulator finished with run_status 0


#### 2. Federated P-Tuning
We use the [FedAvg](https://arxiv.org/abs/1602.05629) algorithm to p-tune the model in a federated scenario. First, create and modify the configuration files again. 
This time, we increase the number of FL rounds and decrease the number of local epochs per round to match the federated scenario.

In [8]:
!python3 create_configs.py --job_folder "jobs/gpt_p-tuning_fedavg_345M" --num_clients 3 --aggregation_epochs 1 --num_rounds 50

Created configs for 3 clients and set ROOT_DIR to /workspace/Code/nvflare/nemo_nvflare/integration/nemo/examples/prompt_learning


Next, simulate the federated p-tuning using FedAvg. Here, each client p-tunes for one local epoch before sending their local model updates to the server for aggregation. This is repeated for 50 FL rounds.

In [9]:
from nvflare import SimulatorRunner    

simulator = SimulatorRunner(
    job_folder="jobs/gpt_p-tuning_fedavg_345M",
    workspace="/tmp/nvflare/nemo/gpt_p-tuning_fedavg_345M",
    n_clients=3,
    threads=3
)
run_status = simulator.run()
print("Simulator finished with run_status", run_status)

2023-06-01 18:18:03,497 - SimulatorRunner - INFO - Create the Simulator Server.
2023-06-01 18:18:03,504 - Cell - INFO - server: creating listener on tcp://0:55255
2023-06-01 18:18:03,506 - Cell - INFO - server: created backbone external listener for tcp://0:55255
2023-06-01 18:18:03,507 - ConnectorManager - INFO - 6413: Try start_listener Listener resources: {'secure': False, 'host': 'localhost'}
2023-06-01 18:18:03,508 - nvflare.fuel.f3.sfm.conn_manager - INFO - Connector [CH00002 PASSIVE tcp://0:4457] is starting
2023-06-01 18:18:04,011 - Cell - INFO - server: created backbone internal listener for tcp://localhost:4457
2023-06-01 18:18:04,017 - nvflare.fuel.f3.sfm.conn_manager - INFO - Connector [CH00001 PASSIVE tcp://0:55255] is starting
2023-06-01 18:18:04,215 - nvflare.fuel.hci.server.hci - INFO - Starting Admin Server localhost on Port 59359
2023-06-01 18:18:04,217 - SimulatorRunner - INFO - Deploy the Apps.
2023-06-01 18:18:04,229 - SimulatorRunner - INFO - Create the simulate c

[NeMo W 2023-06-01 18:18:23 experimental:27] Module <class 'nemo.collections.nlp.data.language_modeling.megatron.megatron_batch_samplers.MegatronPretrainingRandomBatchSampler'> is experimental, not ready for production and is not fully supported. Use at your own risk.
[NeMo W 2023-06-01 18:18:23 experimental:27] Module <class 'nemo.collections.nlp.models.text_normalization_as_tagging.thutmose_tagger.ThutmoseTaggerModel'> is experimental, not ready for production and is not fully supported. Use at your own risk.
[NeMo W 2023-06-01 18:18:23 experimental:27] Module <class 'nemo.collections.nlp.data.language_modeling.megatron.megatron_batch_samplers.MegatronPretrainingRandomBatchSampler'> is experimental, not ready for production and is not fully supported. Use at your own risk.
[NeMo W 2023-06-01 18:18:23 experimental:27] Module <class 'nemo.collections.nlp.models.text_normalization_as_tagging.thutmose_tagger.ThutmoseTaggerModel'> is experimental, not ready for production and is not fully

2023-06-01 18:18:25,109 - ServerRunner - INFO - [identity=simulator_server, run=simulate_job, wf=share_config, peer=site-2, peer_run=simulate_job, task_name=share_config, task_id=6836e3dd-68ed-4928-b9f4-b1b659918590]: assigned task to client site-2: name=share_config, id=6836e3dd-68ed-4928-b9f4-b1b659918590
2023-06-01 18:18:25,111 - ServerRunner - INFO - [identity=simulator_server, run=simulate_job, wf=share_config, peer=site-2, peer_run=simulate_job, task_name=share_config, task_id=6836e3dd-68ed-4928-b9f4-b1b659918590]: sent task assignment to client. client_name:site-2 task_id:6836e3dd-68ed-4928-b9f4-b1b659918590
2023-06-01 18:18:25,115 - GetTaskCommand - INFO - return task to client.  client_name: site-2  task_name: share_config   task_id: 6836e3dd-68ed-4928-b9f4-b1b659918590  sharable_header_task_id: 6836e3dd-68ed-4928-b9f4-b1b659918590
2023-06-01 18:18:25,152 - ServerRunner - INFO - [identity=simulator_server, run=simulate_job, wf=share_config, peer=site-2, peer_run=simulate_job]:

[NeMo W 2023-06-01 18:18:25 experimental:27] Module <class 'nemo.collections.asr.modules.audio_modules.SpectrogramToMultichannelFeatures'> is experimental, not ready for production and is not fully supported. Use at your own risk.
I0601 18:18:25.103726 139825093494592 fl_component.py:134] [identity=site-2, run=simulate_job]: Initializing the Learner...
I0601 18:18:25.104350 139825093494592 fl_component.py:134] [identity=site-2, run=simulate_job]: Running with distributed environment: LOCAL_RANK: 0, RANK: 0, WORLD_SIZE 1, MASTER_ADDR: localhost, and MASTER_PORT: 40037
I0601 18:18:25.104617 139825093494592 fl_component.py:134] [identity=site-2, run=simulate_job]: client runner started
I0601 18:18:25.104791 139825093494592 simulator_worker.py:85] Initialize ClientRunner for client: site-2
I0601 18:18:25.117143 139823787386624 communicator.py:200] Received from simulator_server server  (3492 Bytes). getTask: share_config time: 0.009407520294189453 seconds
I0601 18:18:25.121602 139825093494

2023-06-01 18:18:25,231 - ServerRunner - INFO - [identity=simulator_server, run=simulate_job, wf=share_config, peer=site-1, peer_run=simulate_job, task_name=share_config, task_id=f4db6818-240b-4d93-94ba-19ec7015715a]: assigned task to client site-1: name=share_config, id=f4db6818-240b-4d93-94ba-19ec7015715a
2023-06-01 18:18:25,233 - ServerRunner - INFO - [identity=simulator_server, run=simulate_job, wf=share_config, peer=site-1, peer_run=simulate_job, task_name=share_config, task_id=f4db6818-240b-4d93-94ba-19ec7015715a]: sent task assignment to client. client_name:site-1 task_id:f4db6818-240b-4d93-94ba-19ec7015715a
2023-06-01 18:18:25,234 - GetTaskCommand - INFO - return task to client.  client_name: site-1  task_name: share_config   task_id: f4db6818-240b-4d93-94ba-19ec7015715a  sharable_header_task_id: f4db6818-240b-4d93-94ba-19ec7015715a
2023-06-01 18:18:25,271 - ServerRunner - INFO - [identity=simulator_server, run=simulate_job, wf=share_config, peer=site-1, peer_run=simulate_job]:

...

2023-06-01 18:19:37,731 - ServerRunner - INFO - [identity=simulator_server, run=simulate_job, wf=scatter_and_gather, peer=site-2, peer_run=simulate_job]: got result from client site-2 for task: name=train, id=bf2d9558-29de-4f0e-ade0-e5f9e2a1d9ff
2023-06-01 18:19:37,734 - ServerRunner - INFO - [identity=simulator_server, run=simulate_job, wf=scatter_and_gather, peer=site-2, peer_run=simulate_job, peer_rc=OK, task_name=train, task_id=bf2d9558-29de-4f0e-ade0-e5f9e2a1d9ff]: ignored result submission since server runner's status is done
2023-06-01 18:19:37,736 - SubmitUpdateCommand - INFO - submit_update process. client_name:site-2   task_id:bf2d9558-29de-4f0e-ade0-e5f9e2a1d9ff

2023-06-01 18:19:37,643 - PromptLearner - INFO - [identity=site-2, run=simulate_job, peer=simulator_server, peer_run=simulate_job, task_name=train, task_id=bf2d9558-29de-4f0e-ade0-e5f9e2a1d9ff]: Computed 7 weight differences for global model of length 7
2023-06-01 18:19:37,644 - PromptLearner - INFO - [identity=site

I0601 18:19:37.643822 139825093494592 fl_component.py:134] [identity=site-2, run=simulate_job, peer=simulator_server, peer_run=simulate_job, task_name=train, task_id=bf2d9558-29de-4f0e-ade0-e5f9e2a1d9ff]: Computed 7 weight differences for global model of length 7
I0601 18:19:37.644314 139825093494592 fl_component.py:134] [identity=site-2, run=simulate_job, peer=simulator_server, peer_run=simulate_job, task_name=train, task_id=bf2d9558-29de-4f0e-ade0-e5f9e2a1d9ff]: Local steps per epoch: 9
I0601 18:19:37.644602 139825093494592 fl_component.py:134] [identity=site-2, run=simulate_job, peer=simulator_server, peer_run=simulate_job, task_name=train, task_id=bf2d9558-29de-4f0e-ade0-e5f9e2a1d9ff]: Local epochs finished. Returning shareable
I0601 18:19:37.645701 139825093494592 fl_component.py:134] [identity=site-2, run=simulate_job, peer=simulator_server, peer_run=simulate_job, task_name=train, task_id=bf2d9558-29de-4f0e-ade0-e5f9e2a1d9ff]: finished processing task
I0601 18:19:37.648889 139814

2023-06-01 18:19:39,748 - ServerRunner - INFO - [identity=simulator_server, run=simulate_job, wf=scatter_and_gather, peer=site-2, peer_run=simulate_job]: server runner is finalizing - asked client to end the run
2023-06-01 18:19:39,760 - GetTaskCommand - INFO - return task to client.  client_name: site-2  task_name: __end_run__   task_id:   sharable_header_task_id: 
2023-06-01 18:19:39,767 - FederatedClient - INFO - Shutting down client run: site-1
2023-06-01 18:19:39,769 - FederatedClient - INFO - Shutting down client run: site-2
2023-06-01 18:19:39,772 - FederatedClient - INFO - Shutting down client run: site-3
2023-06-01 18:19:39,772 - ServerRunner - INFO - [identity=simulator_server, run=simulate_job, wf=scatter_and_gather]: asked to abort - triggered abort_signal to stop the RUN
2023-06-01 18:19:39,766 - FederatedClient - INFO - pull_task completed. Task name:__end_run__ Status:True 
2023-06-01 18:19:39,766 - ClientRunner - INFO - [identity=site-2, run=simulate_job, peer=simulator

I0601 18:19:39.766087 139825093494592 fed_client.py:91] pull_task completed. Task name:__end_run__ Status:True 
I0601 18:19:39.766475 139825093494592 fl_component.py:134] [identity=site-2, run=simulate_job, peer=simulator_server, peer_run=simulate_job]: server asked to end the run
I0601 18:19:39.766636 139825093494592 simulator_worker.py:102] End the Simulator run.
I0601 18:19:39.767145 139825093494592 simulator_worker.py:125] Clean up ClientRunner for : site-2 


2023-06-01 18:19:43,291 - MPM - INFO - MPM: Good Bye!
Simulator finished with run_status 0


You can visualize the training process using TensorBoard

In [ ]:
!tensorboard --logdir /tmp/nvflare/nemo

TensorFlow installation not found - running with reduced feature set.

NOTE: Using experimental fast data loading logic. To disable, pass
    "--load_fast=false" and report issues on GitHub. More details:
    https://github.com/tensorflow/tensorboard/issues/4784

TensorBoard 2.9.0 at http://localhost:6006/ (Press CTRL+C to quit)


## Results
In this scenario, all clients utilize the same validation set, allowing for a direct comparison between the locally p-tuned and federated global models. As anticipated, the FedAvg-trained global model exhibits lower validation loss than the models trained solely on their local datasets. This is because the global model has access to all client datasets and can, consequently, generalize better.

![validation loss](./figs/val_loss.svg)

## Inference

We can use `model.generate()` to run inference after p-tuning the model. 
Let's define some test examples to feed to the p-tuned model to see its predictions.

In [ ]:
test_examples = [
    {"taskname": "sentiment", "sentence": "The products have a low salt and fat content ."},
    {"taskname": "sentiment", "sentence": "The agreement is valid for four years ."},
    {"taskname": "sentiment", "sentence": "Diluted EPS rose to EUR3 .68 from EUR0 .50 ."},
    {"taskname": "sentiment", "sentence": "The company is well positioned in Brazil and Uruguay ."},
    {"taskname": "sentiment", "sentence": "Profit before taxes decreased by 9 % to EUR 187.8 mn in the first nine months of 2008 , compared to EUR 207.1 mn a year earlier ."},
]

Next, we will load the global model.

In [ ]:
import os
import torch
import pytorch_lightning as pl
from nemo_nvflare.fed_megatron_gpt_prompt_learning_model import FedMegatronGPTPromptLearningModel
from nemo_nvflare.utils import load_weights
from omegaconf import OmegaConf
from nemo.collections.nlp.parts.nlp_overrides import NLPDDPStrategy
from pytorch_lightning.plugins.environments import TorchElasticEnvironment

# Load model configuration used by one of the clients
config = OmegaConf.load("jobs/gpt_p-tuning_fedavg_345M/server/config/megatron_gpt_prompt_learning_config.yaml")

# Set GPT model path
config.model.language_model_path = "megatron_gpt_345m.nemo"

# Load task templates
config.model.task_templates = OmegaConf.load("jobs/gpt_p-tuning_fedavg_345M/server/config/task_templates.json")

# Set task that were learned
config.model.new_tasks = ["sentiment"]

# Setup cluster environment parameters
# use torch elastic cluster environment so `create_process_externally` is True
# the launcher is set to None. It will not try to spawn new processes.
# It won't create the misconfiguration error because of the `interactive session`
os.environ["LOCAL_RANK"] = '0'
os.environ["RANK"] = '0'
os.environ["WORLD_SIZE"] = '1'
strategy = NLPDDPStrategy(find_unused_parameters=False, no_ddp_communication_hook=True)
plugins = [TorchElasticEnvironment()]

# Set up the trainer and load the model that was used for p-tuning
trainer = pl.Trainer(plugins=plugins, strategy=strategy, **config.trainer)
model = FedMegatronGPTPromptLearningModel(cfg=config.model, trainer=trainer)
model.init_prompt_encoder()

print("Model initialized", type(model))

Overwrite the prompt encoder with the best global model

In [ ]:
ckpt = torch.load("/tmp/nvflare/nemo/gpt_p-tuning_fedavg_345M/simulate_job/app_server/best_FL_global_model.pt")
global_weights = ckpt["model"]

n_loaded = load_weights(model, global_weights, device=torch.device("cuda:0" if torch.cuda.is_available() else "cpu"))
print(f"Loaded {n_loaded} of {len(global_weights)} weights")

Run the model

In [ ]:
response = model.generate(inputs=test_examples, length_params=None)

print('The prediction results of some sample queries with the trained model:')
for result in response['sentences']:
    print(result)
    print("-" * 30)

The expected output predictions look something like this

>      The products have a low salt and fat content . sentiment: neutral
>      ------------------------------
>      The agreement is valid for four years . sentiment: neutral
>      ------------------------------
>      Diluted EPS rose to EUR3 .68 from EUR0 .50 . sentiment: positive
>      ------------------------------
>      The company is well positioned in Brazil and Uruguay . sentiment: positive
>      ------------------------------
>      Profit before taxes decreased by 9 % to EUR 187.8 mn in the first nine months of 2008 , compared to EUR 207.1 mn a year earlier . sentiment: negative
>      ------------------------------